# VAE Training

## imports

In [ ]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

In [2]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## data

In [3]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## architecture

In [4]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [5]:
vae.encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 encoder_conv_0 (Conv2D)        (None, 28, 28, 32)   320         ['encoder_input[0][0]']          
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 28, 28, 32)   0           ['encoder_conv_0[0][0]']         
                                                                                                  
 encoder_conv_1 (Conv2D)        (None, 14, 14, 64)   18496       ['leaky_re_lu[0][0]']            
                                                                                            

In [6]:
vae.decoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_0 (Conv2DTra  (None, 7, 7, 64)         36928     
 nspose)                                                         
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_1 (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                   

## training

In [7]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

In [8]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [9]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [10]:
#train(self, x_train, batch_size, epochs, run_folder, print_every_n_batches = 100, initial_epoch = 0, lr_decay = 1):

vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Train on 60000 samples


2024-11-10 19:31:46.286540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 19:31:46.992752: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-11-10 19:31:46.992831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 73683 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-11-10 19:31:47.027164: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2024-11-10 19:31:47.091409: W tensorflow/c/c_api

Epoch 1/200


2024-11-10 19:31:48.330854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2024-11-10 19:31:48.900733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


  448/60000 [..............................] - ETA: 4:31 - loss: 223.8112 - vae_r_loss: 223.7625 - vae_kl_loss: 0.0487 

/opt/conda/lib/python3.10/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-11-10 19:31:49.133108: W tensorflow/c/c_api.cc:291] Operation '{name:'activation/Sigmoid' id:341 op device:{requested: '', assigned: ''} def:{{{node activation/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](decoder_conv_t_3/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


59776/60000 [============================>.] - ETA: 0s - loss: 58.4291 - vae_r_loss: 55.1627 - vae_kl_loss: 3.2664
Epoch 1: saving model to run/vae/0002_digits/weights/weights-001-58.41.h5

Epoch 1: saving model to run/vae/0002_digits/weights/weights.h5
60000/60000 [==============================] - 16s 262us/sample - loss: 58.4127 - vae_r_loss: 55.1443 - vae_kl_loss: 3.2684 - lr: 5.0000e-04
Epoch 2/200
59872/60000 [============================>.] - ETA: 0s - loss: 51.5691 - vae_r_loss: 47.5918 - vae_kl_loss: 3.9773
Epoch 2: saving model to run/vae/0002_digits/weights/weights-002-51.57.h5

Epoch 2: saving model to run/vae/0002_digits/weights/weights.h5
60000/60000 [==============================] - 14s 232us/sample - loss: 51.5684 - vae_r_loss: 47.5901 - vae_kl_loss: 3.9783 - lr: 5.0000e-04
Epoch 3/200
59776/60000 [============================>.] - ETA: 0s - loss: 50.1622 - vae_r_loss: 45.8963 - vae_kl_loss: 4.2659
Epoch 3: saving model to run/vae/0002_digits/weights/weights-003-50.16.